# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [65]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [66]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,fujin,47.2600,132.0322,-30.43,95,49,3.25,CN,1674848380
1,thinadhoo,0.5333,72.9333,27.28,75,49,2.70,MV,1674848381
2,methoni,36.8197,21.7049,10.68,78,42,2.09,GR,1674848381
3,cuauhtemoc,28.4167,-106.8667,13.30,16,100,2.55,MX,1674848382
4,santa maria,-29.6842,-53.8069,32.61,52,40,6.17,BR,1674848286


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [81]:
%%capture --no-display

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    size = 10,
    scale = 2,
    color = "City"
)

# Display the map plot
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [72]:
# Narrow down cities that fit criteria and drop any results with null values
reduced_city_data_df = city_data_df.loc[city_data_df["Max Temp"]>20,:]

# Drop any rows with null values
reduced_city_data_df = reduced_city_data_df.dropna()
# Display sample data
reduced_city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
1,thinadhoo,0.5333,72.9333,27.28,75,49,2.70,MV,1674848381
4,santa maria,-29.6842,-53.8069,32.61,52,40,6.17,BR,1674848286
5,galesong,-5.3166,119.3661,24.01,90,100,4.63,ID,1674848382
6,xique-xique,-10.8218,-42.7263,30.26,46,80,0.89,BR,1674848383
10,cap malheureux,-19.9842,57.6142,25.82,90,100,2.98,MU,1674848384


### Step 3: Create a new DataFrame called `hotel_df`.

In [63]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = reduced_city_data_df.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] =""

# Display sample data
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
City_ID,,,,,,,,,,
1,thinadhoo,0.5333,72.9333,27.28,75,49,2.70,MV,1674848381,
4,santa maria,-29.6842,-53.8069,32.61,52,40,6.17,BR,1674848286,
5,galesong,-5.3166,119.3661,24.01,90,100,4.63,ID,1674848382,
6,xique-xique,-10.8218,-42.7263,30.26,46,80,0.89,BR,1674848383,
10,cap malheureux,-19.9842,57.6142,25.82,90,100,2.98,MU,1674848384,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 50000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [84]:
%%capture --no-display

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    size = 10,
    scale = 2,
    color = "City",
    hover_cols=["Hotel Name","Humidity","Country"]
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Humidity,Country)